In [ ]:
import os
import glob
import cv2
import numpy as np
import seaborn as sns
from tensorflow.keras import layers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
import tensorflow as tf

In [ ]:
image_gen = ImageDataGenerator()

In [ ]:
!pip install mlxtend
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
import tensorflow as tf
from tensorflow.keras.applications import resnet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
print("Tensorflow version " + tf.__version__)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix, accuracy_score

from mlxtend.plotting import plot_confusion_matrix

In [ ]:
import pandas as pd
CATEGORIES = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
X = []
y = []
data = pd.read_csv(r'H:\Facial Emotion Recognition\Dataset\FER2013\fer2013.csv')  # Remove the extra double quotes
# Check data shape
data.shape

In [ ]:
data.head(200)

In [ ]:
emotion_map = {0: 'Angry', 1: 'Digust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}
emotion_counts = data['emotion'].value_counts(sort=False).reset_index()
emotion_counts.columns = ['emotion', 'number']
emotion_counts['emotion'] = emotion_counts['emotion'].map(emotion_map)
emotion_counts

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming that emotion_counts is a DataFrame with columns 'emotion' and 'number'

plt.figure(figsize=(6, 4))
sns.barplot(data=emotion_counts, x='emotion', y='number')
plt.title('Class distribution')
plt.ylabel('Number', fontsize=12)
plt.xlabel('Emotions', fontsize=12)
plt.show()

In [ ]:
for index, row in data.iterrows():
    try:
        pixels=np.asarray(list(row['pixels'].split(' ')), dtype=np.uint8)
        img = pixels.reshape((48,48))
        X.append(img)
        y.append(row['emotion'])
    except Exception as e:
        pass

In [ ]:
fig = plt.figure(figsize=(10, 6))

for counter, img in enumerate(X[:12]):
    ax = fig.add_subplot(3, 4, counter + 1)
    ax.imshow(X[counter], cmap='gray')
    plt.title(CATEGORIES[y[counter]])
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)

plt.show()

In [ ]:
X = np.array(X, dtype='float32').reshape(-1, 48, 48)
rgb_X = np.repeat(X[..., np.newaxis], 3, -1)
rgb_X=rgb_X/255.
y = np.asarray(y)
y.shape

In [ ]:
rgb_X.shape

In [ ]:
(X_train, X_val, y_train, y_val) = train_test_split(rgb_X, y,
                                                    test_size=0.2,
                                                    random_state=2022,
                                                    shuffle=True,
                                                    stratify=y)

In [ ]:
X_train.shape

In [ ]:
aug_train = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

generator_val = ImageDataGenerator()

In [ ]:
aug_train.fit(X_train)

generator_val.fit(X_val)

In [ ]:
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(CATEGORIES), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
# Checkpoint to save the best model weights
checkpoint = ModelCheckpoint("emotion_detection_densenet.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

# Reduce learning rate if validation loss plateaus
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)

callbacks = [checkpoint, early_stopping, reduce_lr]


In [ ]:
model.compile(optimizer=Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    aug_train.flow(X_train, y_train, batch_size=64),
    validation_data=generator_val.flow(X_val, y_val, batch_size=64),
    steps_per_epoch=len(X_train) // 64,
    validation_steps=len(X_val) // 64,
    epochs=25,
    callbacks=callbacks
)


In [ ]:
import matplotlib.pyplot as plt
c 
# Plot training & validation accuracy and loss
plt.figure(figsize=(12, 4))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Generate predictions on the validation set
predicted_labels = model.predict(X_val)
predicted_labels = np.argmax(predicted_labels, axis=1)

# Calculate the confusion matrix
cm = confusion_matrix(y_val, predicted_labels)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=CATEGORIES, yticklabels=CATEGORIES)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Make predictions on the validation set
y_pred_probs = model.predict(X_val)

# Convert y_val to one-hot encoded format
y_val_one_hot = np.zeros((len(y_val), len(CATEGORIES)))
y_val_one_hot[np.arange(len(y_val)), y_val] = 1

# Calculate ROC-AUC Score
roc_auc = roc_auc_score(y_val_one_hot, y_pred_probs, average='macro')

# Calculate Average Precision (AP) Score
ap_score = average_precision_score(y_val_one_hot, y_pred_probs, average='macro')

print(f'ROC-AUC Score: {roc_auc}')
print(f'Average Precision Score: {ap_score}')


In [ ]:
y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = y_val.astype(int)

class_report = classification_report(y_true, y_pred_classes, target_names=CATEGORIES)
print(class_report)


# Hyperparameter tuning and more aggressive data augmentation

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
# Define a learning rate schedule (e.g., using ReduceLROnPlateau)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)

# Hyperparameter tuning: Adjust learning rate and batch size
learning_rate = 0.001  # Start with a reasonable learning rate
batch_size = 64  # Start with a reasonable batch size

# Define a more aggressive data augmentation configuration
aug_train = ImageDataGenerator(
    rotation_range=30,  # Increase the rotation range
    zoom_range=0.3,  # Increase the zoom range
    width_shift_range=0.3,  # Increase the width shift range
    height_shift_range=0.3,  # Increase the height shift range
    shear_range=0.3,  # Increase the shear range
    horizontal_flip=True,
    fill_mode="nearest"
)

# Compile the model with the chosen learning rate
model.compile(optimizer=Adam(lr=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with the chosen batch size and data augmentation
history = model.fit(
    aug_train.flow(X_train, y_train, batch_size=batch_size),
    validation_data=generator_val.flow(X_val, y_val, batch_size=batch_size),
    steps_per_epoch=len(X_train) // batch_size,
    validation_steps=len(X_val) // batch_size,
    epochs=50,
    callbacks=[reduce_lr]  # You can keep the same callbacks
)


In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy and loss
plt.figure(figsize=(12, 4))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Generate predictions on the validation set
predicted_labels = model.predict(X_val)
predicted_labels = np.argmax(predicted_labels, axis=1)

# Calculate the confusion matrix
cm = confusion_matrix(y_val, predicted_labels)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=CATEGORIES, yticklabels=CATEGORIES)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Make predictions on the validation set
y_pred_probs = model.predict(X_val)

# Convert y_val to one-hot encoded format
y_val_one_hot = np.zeros((len(y_val), len(CATEGORIES)))
y_val_one_hot[np.arange(len(y_val)), y_val] = 1

# Calculate ROC-AUC Score
roc_auc = roc_auc_score(y_val_one_hot, y_pred_probs, average='macro')

# Calculate Average Precision (AP) Score
ap_score = average_precision_score(y_val_one_hot, y_pred_probs, average='macro')

print(f'ROC-AUC Score: {roc_auc}')
print(f'Average Precision Score: {ap_score}')

In [ ]:
y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = y_val.astype(int)

class_report = classification_report(y_true, y_pred_classes, target_names=CATEGORIES)
print(class_report)